In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

The commented code below includes saved functions that query the RCSB Protein DataBase. I originally scraped from this site, but ended up switching to UniProt. I've kept the code below, just in case I want to gather corroborating data later on.

In [25]:
# # This function is adapted from Carlos Oliver
# # https://www.rcsb.org/pages/webservices/python3_search_nmr

# def experimental_query(method):
#     url = 'http://www.rcsb.org/pdb/rest/search'

#     query_text = """
# <?xml version="1.0" encoding="UTF-8"?>

# <orgPdbQuery>

# <version>B0907</version>

# <queryType>org.pdb.query.simple.ExpTypeQuery</queryType>

# <description>Experimental Method Search: Experimental Method="""+method+"""</description>

# <mvStructure.expMethod.value>"""+method+"""</mvStructure.expMethod.value>

# </orgPdbQuery>

# """
#     query_helper(query_text)

# def query_helper(query_text):    

#     print("Query: %s" % query_text)
#     print("Querying RCSB PDB REST API...")

#     header = {'Content-Type': 'application/x-www-form-urlencoded'}

#     response = requests.post(url, data=query_text, headers=header)

#     if response.status_code == 200:
#         print("Found %d PDB entries matching query." % len(response.text))
#         print("Matches: \n%s" % response.text)
#     else:
#         print("Failed to retrieve results")

In [26]:
# solid_state = "SOLID-STATE NMR"
# experimental_query(solid_state)

## Data Collection
I customized advanced queries from UniProt to only select human proteins with at least one interactor.  I choose to include protein mass, length, sequence, and interactors as 

In [ ]:
train = pd.read_csv("./train_set.csv")
test = pd.read_csv("./test_set.csv")